In [2]:
# Load Vissim
# Documents here
# C:\Users\Public\Documents\PTV Vision\PTV Vissim 11\Examples Training
# Signal Control files here
# C:\Program Files\PTV Vision\PTV Vissim 11\API\SignalControl_DLLs

# for loading client
import win32com.client as com
import os
# standard libraries 
import numpy as np
import math
import time
from IPython import display
# For Q-function
from collections import defaultdict
# For saving
import datetime
import dill
import pickle

'''
This loads up a Vissim instance 
    -- Required: inpx file , layx file
    -- warning: quite flakey loading but once loaded okay
'''
def Load_Vissim(End_of_simulation = 10000,\
                Quick_Mode=1,\
                Path_to_network = 'C:\\Users\\nwalton\\Dropbox (The University of Manchester)\\ACTIVE\\TMF\\TMF_Raymond\\Raymond_Turing_RL\\Single_Cross',\
                inpx_Filename = 'Single_Cross_StraightMOVA.inpx',\
                #layx_Filename = 'Sin.layx'\
               ):
    Vissim = None
    # Load Vissim
    Vissim = com.gencache.EnsureDispatch("Vissim.Vissim")
    # Load file
    inpx_Filename                = os.path.join(Path_to_network, inpx_Filename)
    flag_read_additionally  = False # you can read network(elements) additionally, in this case set "flag_read_additionally" to true
    Vissim.LoadNet(inpx_Filename, flag_read_additionally)
    # Load a Layout:
    #layx_Filename = os.path.join(Path_to_network, layx_Filename)
   # Vissim.LoadLayout(layx_Filename)
    # Configure non-GUI for training
    Vissim.Simulation.SetAttValue('UseMaxSimSpeed', True)
    Vissim.Simulation.AttValue('UseAllCores')
    Vissim.Graphics.CurrentNetworkWindow.SetAttValue("QuickMode",Quick_Mode)
    # Set a long simulation time
    Vissim.Simulation.SetAttValue('SimPeriod', End_of_simulation)
    return Vissim

# '''
# Set up Vissim and the parameters for optimizing
# (Worth commenting out after set up)
# if ERROR like ''has no attribute 'CLSIDToClassMap' ''
# DELETE folders:
# C:\Users\nwalton\AppData\Local\Temp\gen_py
# C:\Users\nwalton\AppData\Local\Temp\VISSIM
# Then Restart PC...
# '''

# We attempt 5 times to load vissim

Attempts = 5 
for _ in range(Attempts):
    try:
        Vissim \
        = \
        Load_Vissim()
        #Path_to_network = 'C:\\Users\\nwalton\\OneDrive - The Alan Turing Institute\\Documents\\MLforFlowOptimisation\\Vissim\\Single_Cross_Straight',\
        #inpx_Filename = 'Single_Cross_Straight.inpx',\
        #layx_Filename = 'Single_Cross_Straight.layx'\
        #)
        print("Success")
        break
    except:
        print("Fail")

Success


In [7]:
# Get simulation parameters
'''
Set up the parameters of the optimization
'''

'''
makes a dictionary for all signals and thier positions
'''
def Get_Signal_Positions(Signal_Groups):
    Signal_Positions = dict()
    for SG in Signal_Groups:
        for SH in SG.SigHeads:
            Lane = SH.AttValue('Lane')
            Position = SH.AttValue('Pos')
            Signal_Positions[Lane] = Position
    return Signal_Positions

# Lanes for detection
Lane_List = ['3-1','1-1','7-1','5-1']

# which lane signals can be green at the same time
actions = [(1,0,1,0),\
            (0,1,0,1)]
# Define the Q-function
# Q_fn = Q_function(actions)

# round the state space
rounding = 1.
sim_steps = 100 # number of simulation steps before update
# set the load to be light
number_of_inputs = len(Vissim.Net.VehicleInputs)
new_volume = 400
for key in range(1,number_of_inputs+1):
    Vissim.Net.VehicleInputs.ItemByKey(key).SetAttValue('Volume(1)', new_volume)
    
# get the list of signal controllers
Signal_Controller = Vissim.Net.SignalControllers.GetAll()[0]
Signal_Groups = Signal_Controller.SGs.GetAll()
Signal_Positions = Get_Signal_Positions(Signal_Groups)

# These are states and rewards which are global variables 
# Assigned None for now

Q_Size = None # Queue sizes at junctions
delays = dict() # Total delay and change in delay for each vehicle

In [8]:
def Get_Q_Size(Lane_List=None, rounding=None):
    # Loads globals if variables not specfied
    if Lane_List is None :
        Lane_List = globals()['Lane_List']
    if rounding is None :
        rounding = globals()['rounding']
        
    # initialize with zero queues
    Qsum = 0
    Q_sizes = dict.fromkeys(Lane_List)
    for key in Q_sizes.keys():
        Q_sizes[key]=0

    # initialize with zero numbers of non-waiting cars
    nonQsum = 0
    nonQ_sizes = dict.fromkeys(Lane_List)
    for key in nonQ_sizes.keys():
        nonQ_sizes[key]=0

    # get all Q lengths    
    All_Vehicles = Vissim.Net.Vehicles.GetAll() 
    for Veh in All_Vehicles:
        lane = Veh.AttValue('Lane')
        if lane in Lane_List : 
            if Veh.AttValue('InQueue') == 1 :
                Q_sizes[lane] += 1
            else : 
                nonQ_sizes[lane] += 1

    state = []

    for lane in Lane_List :
        state.append(math.ceil(Q_sizes[lane] / rounding))
        
    return tuple(state)

def Get_Total_Queue(Q_Size=None):
    # Use global as default
    if Q_Size is None:
        Q_Size = Get_Q_Size()
        
    return -sum(Q_Size)

In [14]:
Vissim.Net.VehicleInputs.ItemByKey(1).SetAttValue('Volume(1)', 800)
Vissim.Net.VehicleInputs.ItemByKey(2).SetAttValue('Volume(1)', 150)
Vissim.Net.VehicleInputs.ItemByKey(3).SetAttValue('Volume(1)', 800)
Vissim.Net.VehicleInputs.ItemByKey(4).SetAttValue('Volume(1)', 150)

In [17]:
Vissim.Graphics.CurrentNetworkWindow.SetAttValue("QuickMode",True)

Qs = []
for t in range(10000):
    Vissim.Simulation.RunSingleStep()
    if t % 20 == 0:
        Qs.append( Get_Total_Queue())

np.mean(Qs)


-1.224

In [20]:
Vissim.Graphics.CurrentNetworkWindow.SetAttValue("QuickMode",False)

Qs = []
for t in range(1000):
    Vissim.Simulation.RunSingleStep()
    if t % 20 == 0:
        Qs.append( Get_Total_Queue())

np.mean(Qs)

-1.2

In [22]:
Vissim.Graphics.CurrentNetworkWindow.SetAttValue("QuickMode",False)
for t in range(4000):
    Vissim.Simulation.RunSingleStep()